## Домашнее задание №1

### Автор: Сергеев Константин Олегович

### Задание

In [1]:
# Домашнее задание
# Обучить на датасете AudioCaps линейный адаптер для отображения аудио вектора
# в текстовое представление обученного CLIP

# 0. Выбрать аудио енкодер (CNN16, LanguageBind или другой)
# 1. Посчитать аудио-векторы и текстовые векторы
# 2. Создать модель аудио-проекции с линейным слоем
# 3. Обучить проекцию в контрастив схеме с полностью замороженными енкодерми
# на датасете AudioCaps
# 4. Оценить качество полученного CLAP на задаче классификации аудио

### Импортируем библиотеки

In [2]:
import random
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity

### Загружаем датасет AudioCaps

In [3]:
data_folder = '/mnt/sdb/konsergeev/hse/'
df_train = pd.read_csv(data_folder + 'audiocaps/audiocaps_train.tsv', sep='\t')
df_val = pd.read_csv(data_folder + 'audiocaps/audiocaps_val_new.tsv', sep='\t')
df_test = pd.read_csv(data_folder + 'audiocaps/audiocaps_test_new.tsv', sep='\t')
df_test

,uniq_id,audio,text,duration
0,0,audiocaps/audio/test/0.flac,Constant rattling noise and sharp vibrations,10.000000
1,1,audiocaps/audio/test/1.flac,A rocket flies by followed by a loud explosion...,7.384000
2,2,audiocaps/audio/test/2.flac,Humming and vibrating with a man and children ...,9.999937
3,3,audiocaps/audio/test/3.flac,A train running on a railroad track followed b...,10.000000
4,4,audiocaps/audio/test/4.flac,"Food is frying, and a woman talks",10.000000
...,...,...,...,...
958,970,audiocaps/audio/test/970.flac,Whistling a tune,3.517812
959,971,audiocaps/audio/test/971.flac,A long burp ends in a sigh,10.000000
960,972,audiocaps/audio/test/972.flac,An engine idling with light wind,10.000000
961,973,audiocaps/audio/test/973.flac,Man talking and a tapping clicking,10.000000


### Выбираем аудио энкодер: LanguageBind

In [4]:
%%time

# !git clone -qq https://github.com/PKU-YuanGroup/LanguageBind
# !pip install -qq torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1
# !pip install -qq -r 'LanguageBind/requirements.txt'

CPU times: user 3 µs, sys: 7 µs, total: 10 µs
Wall time: 25.5 µs


In [5]:
%%time

from LanguageBind.languagebind import LanguageBindAudio, LanguageBindAudioTokenizer, LanguageBindAudioProcessor


device = 'cuda:0'

pretrained_ckpt = 'LanguageBind/LanguageBind_Audio'
model = LanguageBindAudio.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir').to(device)
tokenizer = LanguageBindAudioTokenizer.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
audio_process = LanguageBindAudioProcessor(model.config, tokenizer)
model.eval()

/home/konsergeev/venv/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/konsergeev/venv/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/home/konsergeev/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 26.3 s, sys: 12 s, total: 38.3 s
Wall time: 12 s


LanguageBindAudio(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elem

### Считаем аудио и текстовые векторы

In [6]:
%%time

# Проверям работу на тестовом примере


data = audio_process(['LanguageBind/assets/audio/0.wav', 'LanguageBind/assets/audio/1.wav'],
                     ["Training a parakeet to climb up a ladder.", 'A lion climbing a tree to catch a monkey.'],
                     return_tensors='pt').to(device)
with torch.no_grad():
    out = model(**data)

print(torch.softmax(out.image_embeds * 100 @ out.text_embeds.T, dim=-1).detach().cpu().numpy())

[[0.9742649  0.02573503]
 [0.02742632 0.97257376]]
CPU times: user 1.53 s, sys: 456 ms, total: 1.99 s
Wall time: 1.23 s


In [7]:
%%time


def get_vectors(audio_ls, text_ls):
    data = audio_process([data_folder + i for i in audio_ls], text_ls, return_tensors='pt').to(device)
    with torch.no_grad():
        out = model(**data)
    return out.image_embeds, out.text_embeds

def get_vectors_batch(audio_ls, text_ls, batch_size=64):
    num_batchs = len(audio_ls) // batch_size + int(len(audio_ls) % batch_size != 0)
    audio_embeds_result = []
    text_embeds_result = []
    for i in tqdm(range(num_batchs)):
        audio_embeds, text_embeds = get_vectors(audio_ls[i*batch_size:(i+1)*batch_size], text_ls[i*batch_size:(i+1)*batch_size])
        audio_embeds_result.extend(audio_embeds)
        text_embeds_result.extend(text_embeds)
    audio_embeds_result = [audio_embed for audio_embed in np.array([audio_embeds.detach().cpu().numpy() for audio_embeds in audio_embeds_result])]
    text_embeds_result = [text_embed for text_embed in np.array([text_embeds.detach().cpu().numpy() for text_embeds in text_embeds_result])]
    return audio_embeds_result, text_embeds_result


audio_vectors, text_vectors = get_vectors_batch(df_train['audio'].tolist(), df_train['text'].tolist(), batch_size=64)
df_train['audio_vectors_base'] = audio_vectors
df_train['text_vectors_base'] = text_vectors
audio_vectors, text_vectors = get_vectors_batch(df_val['audio'].tolist(), df_val['text'].tolist(), batch_size=64)
df_val['audio_vectors_base'] = audio_vectors
df_val['text_vectors_base'] = text_vectors
audio_vectors, text_vectors = get_vectors_batch(df_test['audio'].tolist(), df_test['text'].tolist(), batch_size=64)
df_test['audio_vectors_base'] = audio_vectors
df_test['text_vectors_base'] = text_vectors
df_test.head()

100%|███████████████████████████████████████████| 16/16 [00:33<00:00,  2.12s/it]

CPU times: user 6h 36min 40s, sys: 13min 28s, total: 6h 50min 9s
Wall time: 29min 27s


,uniq_id,audio,text,duration,audio_vectors_base,text_vectors_base
0,0,audiocaps/audio/test/0.flac,Constant rattling noise and sharp vibrations,10.000000,"[0.0148293665, 0.048589807, -0.012676335, 0.00...","[0.04687137, 0.024851963, -0.0068529323, 0.001..."
1,1,audiocaps/audio/test/1.flac,A rocket flies by followed by a loud explosion...,7.384000,"[-0.03300063, 8.1044716e-05, -0.030373499, -0....","[0.010077148, 0.03264648, -0.027249483, 0.0569..."
2,2,audiocaps/audio/test/2.flac,Humming and vibrating with a man and children ...,9.999937,"[0.003863549, 0.026699983, -0.046704978, 0.014...","[0.028773585, -0.003980448, -0.018655604, 0.01..."
3,3,audiocaps/audio/test/3.flac,A train running on a railroad track followed b...,10.000000,"[-0.00015482766, 0.04142474, 0.034263685, 0.00...","[0.07330153, 0.05360144, 0.037333313, -0.02770..."
4,4,audiocaps/audio/test/4.flac,"Food is frying, and a woman talks",10.000000,"[-0.03696345, 0.035356764, -0.059328858, 0.017...","[0.024296572, 0.022711761, 0.0015205223, -0.02..."


### Считаем метрику без дообучения

In [8]:
# Считаем среднее, попал ли соответствующий текст для аудио в топ-1, топ-3 и топ-10 ближайших по косинусной близости

np.random.seed(2)


def classification_score(audio_vectors, text_vectors):
    # Вычисляем матрицу косинусной близости
    similarity_matrix = cosine_similarity(audio_vectors, text_vectors)
    
    # Инициализируем счетчики для метрик
    top_1_count = 0
    top_3_count = 0
    top_10_count = 0
    
    # Для каждой строки в матрице (каждого аудио-вектора)
    for i in range(similarity_matrix.shape[0]):
        # Получаем индексы текстов, отсортированные по убыванию косинусной близости
        sorted_indices = np.argsort(-similarity_matrix[i])
        
        # Проверяем, находится ли правильный текст в топ-1, топ-3 или топ-10
        if i in sorted_indices[:1]:
            top_1_count += 1
        if i in sorted_indices[:3]:
            top_3_count += 1
        if i in sorted_indices[:10]:
            top_10_count += 1
    
    # Рассчитываем метрики
    total_samples = len(audio_vectors)
    top_1_accuracy = top_1_count / total_samples
    top_3_accuracy = top_3_count / total_samples
    top_10_accuracy = top_10_count / total_samples
    
    return top_1_accuracy, top_3_accuracy, top_10_accuracy

# Пример использования
audio_vectors = np.random.rand(100, 128)  # Пример: 100 аудио-векторов размерности 128
text_vectors = np.random.rand(100, 128)   # Пример: 100 текстовых векторов размерности 128

top_1, top_3, top_10 = classification_score(audio_vectors, text_vectors)
print(f"Top-1 Accuracy: {top_1:.2f}")  # должно быть ~0.01
print(f"Top-3 Accuracy: {top_3:.2f}")  # должно быть ~0.03
print(f"Top-10 Accuracy: {top_10:.2f}")  # должно быть ~0.1

Top-1 Accuracy: 0.01
Top-3 Accuracy: 0.03
Top-10 Accuracy: 0.11


In [9]:
%%time

## Считаем для val


top_1, top_3, top_10 = classification_score(np.array([audio_vector for audio_vector in df_val['audio_vectors_base'].values]),
                                            np.array([text_vector for text_vector in df_val['text_vectors_base'].values]))
print(f"Size Val: {len(df_val)}")
print(f"Top-1 Accuracy: {top_1:.2f}")
print(f"Top-3 Accuracy: {top_3:.2f}")
print(f"Top-10 Accuracy: {top_10:.2f}")

Size Val: 495
Top-1 Accuracy: 0.18
Top-3 Accuracy: 0.36
Top-10 Accuracy: 0.62
CPU times: user 553 ms, sys: 1.65 s, total: 2.2 s
Wall time: 41 ms


In [10]:
%%time

## Считаем для test


top_1, top_3, top_10 = classification_score(np.array([audio_vector for audio_vector in df_test['audio_vectors_base'].values]),
                                            np.array([text_vector for text_vector in df_test['text_vectors_base'].values]))
print(f"Size Test: {len(df_test)}")
print(f"Top-1 Accuracy: {top_1:.2f}")
print(f"Top-3 Accuracy: {top_3:.2f}")
print(f"Top-10 Accuracy: {top_10:.2f}")

Size Test: 963
Top-1 Accuracy: 0.12
Top-3 Accuracy: 0.25
Top-10 Accuracy: 0.48
CPU times: user 1.54 s, sys: 6.68 s, total: 8.22 s
Wall time: 132 ms


### Создаём отдельный линейный слой аудио проекции

Хотя в модели LanguageBind_Audio есть отдельный конечный линейный слой visual_projection и наверное максимально правильно было бы прогонять всю модель и обновлять веса только у этого слоя, прогонять опять всю модель заново на каждом примере довольно долго 

Поэтому будем обучать проекцию как отедльную модель, используя уже посчитанные вектора после слоя модели visual_projection, концептуально ожидаемый эффект должен сохраниться

In [11]:
%%time


class AudioProjection(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AudioProjection, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=False)

    def forward(self, x):
        return self.linear(x)

vector_dim = df_train['audio_vectors_base'].values[0].shape[0]
audio_projection = AudioProjection(vector_dim, vector_dim)

CPU times: user 15.8 ms, sys: 220 ms, total: 236 ms
Wall time: 6.77 ms


### Формируем датасет для контрастив обучения аудио-проекции

In [12]:
%%time

# Формируем данные под обучение
# все положительные пары + 3x случайные негативные пары, как любые другие пары


class AudioTextDataset(Dataset):
    def __init__(self, audio_vectors, text_vectors):
        self.audio_vectors = audio_vectors
        self.text_vectors = text_vectors
        self.num_samples = len(audio_vectors)
        self.positive_pairs = [(i, i) for i in range(self.num_samples)]

    def _generate_negative_pair(self):
        while True:  # с while True в 20 разбыстрее, чем для j формировать список без i и применять choice
            i = random.randint(0, self.num_samples - 1)
            j = random.randint(0, self.num_samples - 1)
            if i != j:
                return (i, j)

    def __len__(self):
        return self.num_samples * 4

    def __getitem__(self, idx):
        if idx < len(self.positive_pairs):
            i, j = self.positive_pairs[idx]
            label = 1
        else:
            i, j = self._generate_negative_pair()
            label = -1

        audio_vector = self.audio_vectors[i]
        text_vector = self.text_vectors[j]
        return audio_vector, text_vector, label

# Создание Dataset и DataLoader
train_dataset = AudioTextDataset(df_train['audio_vectors_base'].values, df_train['text_vectors_base'].values)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

CPU times: user 4.33 ms, sys: 4.02 ms, total: 8.35 ms
Wall time: 8.35 ms


### Функция для вычисления метрики

In [13]:
# Датасет только для аудио векторов
class AudioDataset(Dataset):
    def __init__(self, audio_vectors):
        self.audio_vectors = audio_vectors

    def __len__(self):
        return len(self.audio_vectors)

    def __getitem__(self, idx):
        return self.audio_vectors[idx]

# Функция для получения проекций
def get_projections(audio_vectors, model, batch_size=64):
    dataset = AudioDataset(audio_vectors)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    projections = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch_projections = model(batch)
            projections.append(batch_projections.numpy())

    return [projection for projection in np.concatenate(projections, axis=0)]

### Обучаем аудио-проекцию

In [14]:
%%time


criterion = nn.CosineEmbeddingLoss()
optimizer = optim.Adam(audio_projection.parameters(), lr=0.001)
        
# Обучение модели
num_epochs = 20
for epoch in range(num_epochs):
    audio_projection.train()
    total_loss = 0
    for audio, text, label in tqdm(train_dataloader):

        optimizer.zero_grad()
        output = audio_projection(audio)
        loss = criterion(output, text, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Вычисление метрик на валидационном наборе
    audio_projection.eval()
    projections = get_projections(df_val['audio_vectors_base'].values, audio_projection, batch_size=64)
    texts_vectors = np.array([text_vector for text_vector in df_val['text_vectors_base'].values])
    top_1_val, top_3_val, top_10_val = classification_score(projections, texts_vectors)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_dataloader)}')
    print(f'Val - Top 1: {top_1_val}, Top 3: {top_3_val}, Top 10: {top_10_val}')

100%|██████████████████████████████████████| 3094/3094 [00:26<00:00, 115.03it/s]


Epoch 1/20, Loss: 0.19592139658258664
Val - Top 1: 0.20404040404040405, Top 3: 0.4, Top 10: 0.696969696969697


100%|██████████████████████████████████████| 3094/3094 [00:26<00:00, 115.82it/s]


Epoch 2/20, Loss: 0.18928719050435766
Val - Top 1: 0.18383838383838383, Top 3: 0.3838383838383838, Top 10: 0.7111111111111111


100%|██████████████████████████████████████| 3094/3094 [00:26<00:00, 117.35it/s]


...

Epoch 18/20, Loss: 0.183423142836856
Val - Top 1: 0.20606060606060606, Top 3: 0.42424242424242425, Top 10: 0.7535353535353535


100%|██████████████████████████████████████| 3094/3094 [00:26<00:00, 118.03it/s]


Epoch 19/20, Loss: 0.1831100916016772
Val - Top 1: 0.2222222222222222, Top 3: 0.397979797979798, Top 10: 0.7333333333333333


100%|██████████████████████████████████████| 3094/3094 [00:26<00:00, 118.59it/s]


Epoch 20/20, Loss: 0.18317663471762913
Val - Top 1: 0.2222222222222222, Top 3: 0.41414141414141414, Top 10: 0.7292929292929293
CPU times: user 6h 47min 52s, sys: 7min 56s, total: 6h 55min 48s
Wall time: 8min 50s


### Считаем конечные вектора и сохраняем проекцию

In [15]:
%%time


audio_projection.eval()

# Получаем проекции
df_val['audio_vectors_tuning'] = get_projections(df_val['audio_vectors_base'].values, audio_projection, batch_size=64)
df_test['audio_vectors_tuning'] = get_projections(df_test['audio_vectors_base'].values, audio_projection, batch_size=64)
torch.save(audio_projection.state_dict(), 'audio_projection_weights.pth')
df_test.head()

CPU times: user 3.87 s, sys: 3.36 s, total: 7.23 s
Wall time: 88.3 ms


,uniq_id,audio,text,duration,audio_vectors_base,text_vectors_base,audio_vectors_tuning
0,0,audiocaps/audio/test/0.flac,Constant rattling noise and sharp vibrations,10.000000,"[0.0148293665, 0.048589807, -0.012676335, 0.00...","[0.04687137, 0.024851963, -0.0068529323, 0.001...","[-0.21079339, 0.32035896, -0.26620892, -0.1130..."
1,1,audiocaps/audio/test/1.flac,A rocket flies by followed by a loud explosion...,7.384000,"[-0.03300063, 8.1044716e-05, -0.030373499, -0....","[0.010077148, 0.03264648, -0.027249483, 0.0569...","[0.1735703, -0.51006633, -0.18793182, -0.18161..."
2,2,audiocaps/audio/test/2.flac,Humming and vibrating with a man and children ...,9.999937,"[0.003863549, 0.026699983, -0.046704978, 0.014...","[0.028773585, -0.003980448, -0.018655604, 0.01...","[0.10546263, 0.27364796, -0.17369564, -0.07990..."
3,3,audiocaps/audio/test/3.flac,A train running on a railroad track followed b...,10.000000,"[-0.00015482766, 0.04142474, 0.034263685, 0.00...","[0.07330153, 0.05360144, 0.037333313, -0.02770...","[0.70919085, 0.75149995, 0.54273933, -0.503389..."
4,4,audiocaps/audio/test/4.flac,"Food is frying, and a woman talks",10.000000,"[-0.03696345, 0.035356764, -0.059328858, 0.017...","[0.024296572, 0.022711761, 0.0015205223, -0.02...","[0.14684911, 0.18128146, 0.088608846, 0.116422..."


### Считаем метрику с дообучением

In [16]:
%%time


## Считаем для val


top_1, top_3, top_10 = classification_score(np.array([audio_vector for audio_vector in df_val['audio_vectors_tuning'].values]),
                                            np.array([text_vector for text_vector in df_val['text_vectors_base'].values]))
print(f"Size Val: {len(df_val)}")
print(f"Top-1 Accuracy: {top_1:.2f}")
print(f"Top-3 Accuracy: {top_3:.2f}")
print(f"Top-10 Accuracy: {top_10:.2f}")

Size Val: 495
Top-1 Accuracy: 0.22
Top-3 Accuracy: 0.41
Top-10 Accuracy: 0.73
CPU times: user 210 ms, sys: 1.74 s, total: 1.95 s
Wall time: 35.9 ms


In [17]:
%%time


## Считаем для test


top_1, top_3, top_10 = classification_score(np.array([audio_vector for audio_vector in df_test['audio_vectors_tuning'].values]),
                                            np.array([text_vector for text_vector in df_test['text_vectors_base'].values]))
print(f"Size Test: {len(df_test)}")
print(f"Top-1 Accuracy: {top_1:.2f}")
print(f"Top-3 Accuracy: {top_3:.2f}")
print(f"Top-10 Accuracy: {top_10:.2f}")

Size Test: 963
Top-1 Accuracy: 0.17
Top-3 Accuracy: 0.35
Top-10 Accuracy: 0.62
CPU times: user 1.21 s, sys: 6 s, total: 7.21 s
Wall time: 113 ms


### Итог

С дообоучением линейной аудио-проекции метрика выросла на тесте:
- top_1: c 0.12 до 0.17
- top_3: c 0.25 до 0.35
- top_10: c 0.48 до 0.62

При размере выборки: 963 пары

Дообучение линейной аудио-проекции на своих данных занимает не так много ресурсов, но при этом даёт отличные результаты.

Неожиданные для меня офф-топ инсайты полученные во время работы:
- с яндекс диска нельзя скачать файл с помошью wget, нужно использовать api
- .tsv, это тот же .csv, только с sep='\t'
- для H100 подходит только CUDA 11.8+ и соответствующая версия torch, поэтому LanguageBind получилось завести только на А40
- генерация негативных пар с while True и проверкой i != j работает в 20 разбыстрее, чем для j формировать список без i и применять choice
- генерация случайной негативной пары при каждом взятии работает лучше, чем заранее задать все негативные пары, так как при нескольких эпохах получаем эффективную выборку больше